In [1]:
import numpy as np
import pandas as pd
import neptune
import os
import datetime
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

n_embed_features = 10
n_features = 1
timesteps = 30

# define model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1, clipvalue=0.5)

input_layer_embed = tf.keras.Input(shape=(n_embed_features,))
input_layer_lstm = tf.keras.Input(shape=(n_features, timesteps,))

x = tf.keras.layers.Embedding(input_dim=n_embed_features, output_dim=2)(input_layer_embed)
x = tf.keras.layers.Reshape((1, 2))(x)
x = tf.keras.Model(inputs=input_layer_embed, outputs=x)

y = tf.keras.layers.InputLayer()(input_layer_lstm)
y = tf.keras.Model(inputs=input_layer_lstm, outputs=y)

combined = tf.keras.layers.concatenate([x.output, y.output])

#z = tf.keras.layers.Reshape((n_steps, n_features))
z = tf.keras.layers.Reshape((timesteps, n_features + 2))(combined)
z = tf.keras.layers.LSTM(50, activation='relu')(z)

z = tf.keras.Model(inputs=[x.input, y.input], outputs=z)

z.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'mse'])
z.summary()
history = z.fit(X, y, validation_data=[X_val, y_val], epochs=2000, verbose=2)

#model.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'mse'])

#model.summary()

#history = model.fit(X, y, validation_data=[X_val, y_val], epochs=2000, verbose=2)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 2)        20          input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1, 30)]      0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 2)         0           embedding[0][0]                  
____________________________________________________________________________________________

NameError: name 'X' is not defined

In [3]:
from keras.utils.vis_utils import plot_model

plot_model(z, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Using TensorFlow backend.


OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.